# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
x_data = housing.drop('medianHouseValue', axis=1)

In [5]:
labels = housing['medianHouseValue']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x_data,
                                                    labels,
                                                    test_size=0.33)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()

In [10]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [12]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [13]:
import tensorflow as tf

In [14]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [15]:
feat_cols = [age, rooms, bedrooms, pop, households,income]

In [16]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [17]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/g1/nd73qdpj6kjd99k5hl46g6r00000gn/T/tmp_rm8m5_r', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [18]:
model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/g1/nd73qdpj6kjd99k5hl46g6r00000gn/T/tmp_rm8m5_r/model.ckpt.
INFO:tensorflow:loss = 383718980000.0, step = 1
INFO:tensorflow:global_step/sec: 771.735
INFO:tensorflow:loss = 555028450000.0, step = 101 (0.133 sec)
INFO:tensorflow:global_step/sec: 788.326
INFO:tensorflow:loss = 679405360000.0, step = 201 (0.128 sec)
INFO:tensorflow:global_step/sec: 782.785
INFO:tensorflow:loss = 685650100000.0, step = 301 (0.125 sec)
INFO:tensorflow:global_step/sec: 802.432
INFO:tensorflow:loss = 536883720000.0, step = 401 (0.125 sec)
INFO:tensorflow:global_step/sec: 740.698
INFO:tensorflow:loss = 502603250000.0, step = 501 (0.134 sec)
INFO:tensorflow:global_step/sec: 762.468
INFO:tensorflow:loss = 552120100000.0, step = 601 (0.132 sec)
INFO:tensorflow:global_step/sec: 724.59
INFO:tensorflow:loss = 410347730000.0, step = 701 (0.137 sec)
INFO:tensorflow:global_step/sec: 770.109
INFO:tensorflow:loss = 45155

##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [20]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)

In [21]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/g1/nd73qdpj6kjd99k5hl46g6r00000gn/T/tmp_rm8m5_r/model.ckpt-20000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test,final_preds)**0.5

98207.3476938669

# Great Job!